In [27]:
%%capture
import matplotlib.pyplot as plt
import numpy as np
import math
import pandas as pd
import re
import seaborn as sns
import nltk 
import heapq
from nltk import RegexpTokenizer as rpt
from nltk.corpus import stopwords as sw
from string import punctuation 

nltk.download('punkt')
nltk.download('stopwords')
stopwords = sw.words('portuguese')

data_url="https://raw.githubusercontent.com/liraop/recinfo_lab2/master/data/results.csv"
data = pd.read_csv(data_url).replace(np.nan, '', regex=True)
documents = data.text.count()
N = documents

def parse(text):
    words = []
    word_pattern = rpt(r'\w+')
    year_pattern = rpt(r'\d{4}')
    
    patterns = [word_pattern, year_pattern]
    
    for pattern in patterns:
        tokens = []
        for token in pattern.tokenize(text):
            if token not in stopwords and len(token) > 3:
                tokens.append(token)
        words.extend(tokens)
    return words


def build_index(dataset):
    document_index = 0
    index = {"doc_row": []}
    
    for entry in dataset.text:
        document_index = document_index + 1
        index["doc_row"].append(document_index)
            
        for ngram in parse(entry):
                if ngram in index: #is ngram already on index?
                    if document_index in index[ngram]: # is it in the same document?
                        index[ngram][document_index] = index[ngram][document_index] + 1
                    else: # nope
                        index[ngram][document_index] = 1 
                else: # no, sir
                    index[ngram] = {document_index: 1}
    
    return index
                        
index = build_index(data)

def get_inter_ab(index, word1, word2):
    docs = 0.0
    
    if word1 not in index or word2 not in index:
        return 0.0
    
    word1_inverted_list = index[word1].keys()
    word2_inverted_list = index[word2].keys()
    
    for doc_id in word1_inverted_list:
        if doc_id in word2_inverted_list:
            docs += 1.0
    
    return docs

def get_n_word(index,word):
    if word in index:
        inverted_list = index[word]
        return float(len(inverted_list))
    else:
        return 0.0

#### Explicar aqui as funções acima

In [28]:
def dice(index, word1, word2):
    n_a = get_n_word(index,word1)
    n_b = get_n_word(index,word2)
    n_ab = get_inter_ab(index,word1,word2)
    
    dom = n_a + n_b            
    
    if dom == 0.0:
        return 0
    else:
        return n_ab/dom

def emim(index, word1, word2):
    n_a = get_n_word(index,word1)
    n_b = get_n_word(index,word2)
    n_ab = get_inter_ab(index,word1,word2)
    
    
    dom = n_a * n_b
    factor = 0
    
    if (dom != 0):
        factor = N * (n_ab/dom)
        
    if factor == 0: return 0
       
    return n_ab * math.log10(factor)


def x2(index, word1, word2):
    n_a = get_n_word(index,word1)
    n_b = get_n_word(index,word2)
    n_ab = get_inter_ab(index,word1,word2)
    
    dom = n_a * n_b
    
    if (dom == 0): return 0
    
    num = math.pow((n_ab - (1/N)*n_a*n_b),2)
    
    return num/dom
    
def mim(index, word1, word2):
    n_a = get_n_word(index,word1)
    n_b = get_n_word(index,word2)
    n_ab = get_inter_ab(index,word1,word2)
    
    dom = n_a * n_b            
    
    if dom == 0.0:
        return 0
    else:
        return n_ab/dom
    
    
def get_query_top10rank(index, query, metric):
    lst=[]
    for word in index.keys():
        if word != 'doc_row' and word != query:
          lst.append([word, metric(index, query, word)])
    
    df_tmp = pd.DataFrame(lst, columns=["word", "metric"])
    df_tmp['r']= df_tmp.metric.rank(ascending=False, method="first")
    df_tmp.sort_values("r", inplace = True)
    
    y = []
    for word in df_tmp[:10].word:
        y.append(word)
        
    return y

#### Calcule as top-10 palavras mais associadas a cada uma dessas 5 palavras de acordo com as 4 métricas que vimos na aula. Você deve produzir uma tabela similar à tabela 6.3 do capítulo 6 do livro texto (pág. 204). Qual métrica você acha que obteve os melhores resultados? Por que? (20 pts.)

In [29]:
queries = ["federal","Federal","governo","seleção","presidente"]
methods = [mim, emim, x2, dice]
dataframes = []

i = 0
for query in queries:
    df = pd.DataFrame(columns=["MIM","EMIM","X2","DICE"])
    for i in range(4):
        i += 1
        if (i == 1):
            df["MIM"] = get_query_top10rank(index, query, mim)
        elif (i == 2):
            df["EMIM"] = get_query_top10rank(index, query, emim)
        elif (i == 3):
            df["X2"] = get_query_top10rank(index, query, x2)
        else:
            df["DICE"] = get_query_top10rank(index, query, dice)
    
    dataframes.append(df)

### Query: federal 

In [30]:
dataframes[0]

,MIM,EMIM,X2,DICE
0,juíza,Bolsonaro,deputado,deputado
1,Ivani,deputado,Institucional,Bolsonaro
2,instaurou,afirmou,Orçamento,Planalto
3,citando,Federal,Cláudio,afirmou
4,rememorar,Planalto,Heleno,Federal
5,alusiva,militares,eleita,Constituição
6,compatível,Paulo,Osmar,militares
7,desobedece,Constituição,Planalto,Paulo
8,magistrada,chefe,Bolsonaro,chefe
9,comemorativas,União,execução,União


#### Deliberadamente escolhida em minúscula, para diferenciar da próxima query em termos semânticos, nesta expensão a métrica `EMIM`, apesar de sugerir os mesmos termos que `DICE`, sugere termos mais próximos considerando o rank, visto que federal normalmente está relacionado a cargos políticos e referências ao governo. 

### Query: Federal

In [31]:
dataframes[1]

,MIM,EMIM,X2,DICE
0,juíza,Ministério,Ministério,Ministério
1,Ivani,Público,Público,Público
2,instaurou,Bolsonaro,Temer,Bolsonaro
3,rememorar,Temer,Vieira,1964
4,alusiva,1964,retomada,corrupção
5,compatível,corrupção,Michel,Temer
6,desobedece,caso,Lava,caso
7,Interamericana,federal,Jato,federal
8,apurar,Brasil,1964,Paulo
9,magistrada,institucional,corrupção,ditadura


#### Federal normalmente se relaciona a instituições ou órgãos governamentais, por isso a letra maiúscula. Neste sentido, vemos que `EMIM` e `DICE` retornam um conjunto de palavras similares, sendo o primeiro mais próximo de palavras atuais.

### Query: governo

In [32]:
dataframes[2]

,MIM,EMIM,X2,DICE
0,enriquecer,país,governos,governos
1,controversa,governos,Pinochet,político
2,didáticos,político,cientista,cientista
3,Fabian,Bolsonaro,vivendo,oposição
4,Salvioli,Governo,didáticos,Bolsonaro
5,Reparação,cientista,historicamente,país
6,reverbera,oposição,chileno,afirmou
7,historiografia,afirmou,assassinar,educação
8,Napolitano,brasileiro,PSOL,brasileiro
9,negacionismo,Estado,democrático,democracia


####  Aqui temos um caso mais claro que `EMIM` traz sugestões mais próximas do que esperamos de acordo com a base de textos. 

### Query: seleção

In [33]:
dataframes[3]

,MIM,EMIM,X2,DICE
0,contida,treinador,treinador,treinador
1,incrédulo,futebol,competições,futebol
2,reescreveriam,jogador,futebol,jogador
3,Sassuolo,competições,confederação,competições
4,alucinaram,equipe,Giovanni,temporada
5,Dybala,temporada,amistoso,jogos
6,donna,primeiros,treinadores,jogadores
7,omitindo,jogos,incorporação,Copa
8,narcisista,lugar,demandas,técnico
9,Série,jogadores,certas,confederação


#### Mais um empate claro entre `EMIM` e `DICE`, tendo logo atrás `X2`. Desconsiderando o rank e observando as palavras, `DICE` sugere melhores palavras, isto é, palavras mais relacionadas a `seleção`.

### Query: presidente

In [34]:
dataframes[4]

,MIM,EMIM,X2,DICE
0,juíza,Bolsonaro,Bolsonaro,feira
1,Ivani,ministro,ministro,Bolsonaro
2,comemorarem,Governo,deputado,Governo
3,instaurou,feira,Governo,sobre
4,citando,político,secretário,ainda
5,rememorar,deputado,político,país
6,alusiva,afirmou,Câmara,ministro
7,Março,apoio,Twitter,anos
8,compatível,militar,vice,disse
9,promovido,disse,militares,Brasil


#### `X2` sugere as melhores palavras relacionadas a query. Inclusive `Twitter`. 

#### De acordo com a métrica que deu os melhores resultados na sua opinião, execute agora cada consulta (usando a abordagem documento- ou termo-por-vez)  expandido-a com: os top-3, top-5 e top-10 documentos. O que acontece com a qualidade dos resultados em cada caso? Aumenta ou diminui? Justifique bem sua resposta. (25 pts.)